# Project Overview

This project explores the strategic implications of fuel prices in Brazil using data from the National Agency of Petroleum, Natural Gas and Biofuels (ANP). Fuel price fluctuations directly impact transportation, logistics, and consumer costs, making it essential for policymakers and businesses to understand their trends and regional disparities.

The primary goal is to develop predictive machine learning models capable of forecasting average fuel prices based on historical trends and contextual variables such as region, fuel type, and economic indicators. This analysis supports strategic decision-making, enabling stakeholders to anticipate price changes and design better pricing policies or subsidy programs.

By applying data science techniques, including exploratory data analysis, feature engineering, and supervised learning algorithms, this project aims to:

- Identify regional and temporal patterns in fuel prices;
- Evaluate and compare machine learning models for price prediction;
- Provide data-driven insights and strategic recommendations;
- Highlight potential challenges in the Brazilian fuel market.

This project is part of the Strategic Thinking module at CCT College Dublin and integrates technical, business, and policy-driven perspectives.

# Project Plan and Timeline

To ensure a structured approach, the project followed a defined plan with clearly segmented phases. Each phase included specific goals and deliverables, from data understanding to machine learning implementation and final reporting.

| Phase                        | Description                                                         | Timeline      |
|-----------------------------|----------------------------------------------------------------------|---------------|
| 1. Problem Definition       | Define business objective, research question, and hypotheses         | Week 1        |
| 2. Data Understanding       | Load, inspect, clean, and describe the dataset                       | Week 1-2      |
| 3. Exploratory Data Analysis (EDA) | Perform descriptive statistics and visualize key patterns         | Week 2-3      |
| 4. Feature Engineering      | Create new variables, encode categoricals, handle dates              | Week 3        |
| 5. Model Development        | Implement machine learning models (Linear, RF, MLP)                  | Week 4-5      |
| 6. Hyperparameter Tuning    | Optimize models using GridSearchCV with cross-validation             | Week 5-6      |
| 7. Results Evaluation       | Compare model metrics, generate strategic insights                   | Week 6        |
| 8. Documentation            | Finalize notebook, write report, create poster, and prepare GitHub   | Week 7        |

This timeline allowed for gradual development while ensuring room for iteration, model refinement, and integration of strategic insights.


# Load and Inspect Dataset

In [ ]:
# Libraries

# Data exploration
import pandas as pd
import numpy as np

# To create graphics
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns

# Machine Learning preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Train/test data splitting
from sklearn.model_selection import train_test_split

# Handle class imbalance
from sklearn.utils import class_weight

# Random operations
import random

# Deep Learning framework
import tensorflow as tf

# Format numerical outputs
pd.options.display.float_format = '{:.2f}'.format


In [2]:
# Load the dataset
df = pd.read_excel("Gas_prices_Brazil.xlsx")

# Display basic information
display("Shape of the dataset:", df.shape)
display("\nData types and non-null counts:")
display(df.info())

# Preview the first 5 rows
df.head()

'Shape of the dataset:'

(58188, 17)

'\nData types and non-null counts:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58188 entries, 0 to 58187
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   START DATE                       58188 non-null  datetime64[ns]
 1   REGION                           58188 non-null  object        
 2   STATE                            58188 non-null  object        
 3   PRODUCT                          58188 non-null  object        
 4   NUMBER OF STATIONS SURVEYED      58188 non-null  int64         
 5   UNIT OF MEASUREMENT              58188 non-null  object        
 6   AVERAGE RESALE PRICE             58188 non-null  float64       
 7   STANDARD DEVIATION RESALE        58188 non-null  float64       
 8   MINIMUM RESALE PRICE             58188 non-null  float64       
 9   MAXIMUM RESALE PRICE             58188 non-null  float64       
 10  AVERAGE RESALE MARGIN            58188 non-null  float64  

None

,START DATE,REGION,STATE,PRODUCT,NUMBER OF STATIONS SURVEYED,UNIT OF MEASUREMENT,AVERAGE RESALE PRICE,STANDARD DEVIATION RESALE,MINIMUM RESALE PRICE,MAXIMUM RESALE PRICE,AVERAGE RESALE MARGIN,RESALE COEFFICIENT OF VARIATION,AVERAGE PRICE DISTRIBUTION,STANDARD DEVIATION DISTRIBUTION,MINIMUM PRICE DISTRIBUTION,MAXIMUM PRICE DISTRIBUTION,COEF OF VARIATION DISTRIBUTION
0,2018-01-07,WEST CENTER,DISTRITO FEDERAL,HYDRATED ETHANOL,39,R$/l,3.431,0.127,3.199,3.599,0.265,0.037,3.166,0.056,3.0717,3.2617,0.018
1,2018-01-07,WEST CENTER,GOIAS,HYDRATED ETHANOL,234,R$/l,2.957,0.137,2.750,3.390,0.405,0.046,2.552,0.160,2.3748,3.0462,0.063
2,2018-01-07,WEST CENTER,MATO GROSSO,HYDRATED ETHANOL,156,R$/l,2.548,0.161,2.249,3.090,0.311,0.063,2.237,0.075,2.1000,2.3900,0.034
3,2018-01-07,WEST CENTER,MATO GROSSO DO SUL,HYDRATED ETHANOL,86,R$/l,3.237,0.157,3.088,3.619,0.513,0.049,2.724,0.126,2.5481,3.0050,0.046
4,2018-01-07,WEST CENTER,DISTRITO FEDERAL,REGULAR GASOLINE,47,R$/l,4.173,0.142,3.890,4.600,0.289,0.034,3.884,0.072,3.7339,3.9958,0.019


# Dataset Description

The dataset used in this project was sourced from the Brazilian National Agency of Petroleum, Natural Gas and Biofuels (ANP). It contains detailed monthly records of fuel prices across all Brazilian states and regions.

### 📌 Key Attributes:
- `start_date`: The beginning date of the fuel price collection period.
- `region`: The geographic region in Brazil (e.g., North, South, Southeast).
- `state`: The specific state where prices were recorded.
- `fuel`: Type of fuel (e.g., Gasoline, Diesel, Ethanol, GNV).
- `unit`: Measurement unit (e.g., R$/liter, R$/kg, R$/m³).
- `avg_price`: Average resale price of the fuel.
- `sd_price`: Standard deviation of resale prices.
- `min_price`, `max_price`: Minimum and maximum prices recorded.
- `avg_price_margin`: Difference between resale and distribution price.
- `coef_price`: Coefficient of variation for resale prices.
- `dist_avg_price`: Average distribution price to fuel stations.
- `dist_sd_price`, `dist_min_price`, `dist_max_price`: Stats on distribution prices.
- `coef_dist`: Coefficient of variation for distribution prices.
- `month`, `year`, `month-year`: Extracted temporal features for trend analysis.

### 🧮 Dataset Summary:
- **Time Span:** Monthly data from multiple years (to be confirmed by `.min()` and `.max()`).
- **Observations:** Over 20,000 rows covering all regions and main fuel types.
- **Granularity:** Regional and state-level resolution.
- **Source:** [ANP Open Data](https://www.gov.br/anp/pt-br)

This dataset enables both temporal and spatial analysis of fuel price behavior, as well as predictive modeling through machine learning techniques.
